In [1]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub 
import tensorflow_datasets as tfds

2025-01-29 10:16:04.841259: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 10:16:04.852948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738145764.866817   21145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738145764.870948   21145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 10:16:04.884776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
IMAGE_RES = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label

In [4]:
(raw_train, raw_validation, raw_test) , metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

print(num_examples)
print(num_classes)

BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

for  image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

2025-01-29 10:16:06.923472: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


23262
2


2025-01-29 10:16:07.154401: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


2025-01-29 10:16:09.188496: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


TensorShape([32, 224, 224, 3])

In [14]:
module_selection = ("mobilenet_v2_100_224", 224, 1280)  # Updated module name
handle_base, pixels, FV_SIZE = module_selection

# Use the correct non-preview URL
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4".format(handle_base)

IMAGE_SIZE = (pixels, pixels)

feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=(224, 224, 3),
    trainable=False
)

In [15]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fbba5194190> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [8]:
inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))

x = feature_extractor(inputs,  training=False)  # Output shape: (batch_size, 1280)

x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_1>
  • training=False

In [5]:
# module_selection = ("mobilenet_v2", 224, 1280)
# handle_base, pixels, FV_SIZE = module_selection

# MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

# IMAGE_SIZE = (pixels, pixels)

# print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

# feature_extractor = hub.KerasLayer(
#     MODULE_HANDLE,
#     input_shape = IMAGE_SIZE + (3, ),
#     trainable = False
# )

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [ ]:
# # Define input layer
# inputs = tf.keras.Input(shape=IMAGE_SIZE + (3, ))

# # Pass input through feature extractor
# x = feature_extractor(inputs)

# # Flatten the output of the feature extractor
# x = tf.keras.layers.Flatten()(x)

# # Ensure correct shape (Flatten if necessary)
# x = tf.keras.layers.Dense(256, activation='relu')(x)  # Optional fully connected layer
# x = tf.keras.layers.Dense(num_classes, activation='softmax')(x) # Create a softmax layer

# # Define the model
# model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# create the model 

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(IMAGE_RES, IMAGE_RES, 3)),
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# compiling the model 

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# training the model

hist = model.fit(
    train_batches, 
    epochs=5,
    validation_data=validation_batches
)

In [ ]:
# evaluating the model


In [ ]:
# saving the model
CATS_VS_DOGS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)